# 2 Player FPSB Auction with symmetric unfirorm valuation distributions

## Imports

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [3]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import AuctionEnvironment

In [4]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [5]:
torch.cuda.is_available()

True

## Settings

In [6]:
run_dir = 'fpsb'
run_name = 'test-experiment2'
logdir = os.path.join(root_path, 'notebooks', run_dir , run_name)

In [7]:
## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
u_lo =0
u_hi =10

## Environment settings
#training batch size
batch_size = 2**5
input_length = 1

# model architecture
size_hidden_layer = 20

# optimization params
epoch = 10000
learning_rate = 2e-2
lr_decay = True
lr_decay_every = 2000
lr_decay_factor = 0.8

sigma = .1 #ES noise parameter
n_perturbations = 32

# plot and log training options
plot_epoch = 50
plot_points = min(150, batch_size)
sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [8]:
# for evaluation
def optimal_bid(valuation):
    return valuation / 2

In [9]:
# Wrapper transforming a strategy to bidder, used by the optimizer
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [10]:
model = NeuralNetStrategy(input_length, size_hidden_layer = size_hidden_layer, requires_grad=False).cuda()

In [11]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = AuctionEnvironment(mechanism,
                  agents = [], #dynamically built
                  max_env_size = 1, #
                  batch_size = batch_size,
                  n_players =2,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [12]:
def log_hyperparams(writer):
    writer.add_scalar('hyperparams/batch_size', batch_size)
    writer.add_scalar('hyperparams/size_hidden_layer', size_hidden_layer)
    writer.add_scalar('hyperparams/learning_rate', learning_rate)
    writer.add_scalar('hyperparams/sigma', sigma)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations)
    
    

## Training

In [13]:
torch.cuda.empty_cache()
writer = SummaryWriter(logdir)
log_hyperparams(writer)

for e in range(epoch+1):    
    
    # lr decay?
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        learning_rate = learning_rate * lr_decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
        
    # always: do optimizer step
    utility = -optimizer.step()
    writer.add_scalar('eval/utility', utility, e) 
    
    # plot + eval
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = (b/v).mean()
        writer.add_scalar('eval/utility', utility, e)
        writer.add_scalar('eval/share', share, e)       
            
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f}".format(e, share, utility))                
        
        # subsample points and plot
        v = v.detach().cpu().numpy()[:plot_points]
        b= b.detach().cpu().numpy()[:plot_points]
        optimal = optimal_bid(v)        
        fig = plt.figure()
        plt.plot(v,b, 'o', v, optimal, 'r-')        
        writer.add_figure('eval/bid_function', fig, e)        
        plt.show()
        
        # first step: write model
        if e==0:
            writer.add_graph(model, bidder.valuations)        
    
   
        
torch.cuda.empty_cache()
writer.close()

C:\Users\stefan\Anaconda3\envs\bnelearn\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0: 	avg bid: 0.058996, 	utility: 5.164916
Epoch 50: 	avg bid: 1.015829, 	utility: 1.144553
Epoch 100: 	avg bid: 1.260799, 	utility: 1.143706
Epoch 150: 	avg bid: 1.333381, 	utility: 0.238102
Epoch 200: 	avg bid: 1.530139, 	utility: 0.671148
Epoch 250: 	avg bid: 1.067875, 	utility: 0.090646
Epoch 300: 	avg bid: 1.148338, 	utility: 0.815727
Epoch 350: 	avg bid: 1.355160, 	utility: 0.723101
Epoch 400: 	avg bid: 1.217865, 	utility: 1.057261
Epoch 450: 	avg bid: 1.473626, 	utility: 0.135799
Epoch 500: 	avg bid: 1.181976, 	utility: 0.821988
Epoch 550: 	avg bid: 1.259529, 	utility: 1.180389
Epoch 600: 	avg bid: 1.109935, 	utility: 0.255002
Epoch 650: 	avg bid: 1.320099, 	utility: 0.761274
Epoch 700: 	avg bid: 1.083619, 	utility: 1.314429
Epoch 750: 	avg bid: 1.115850, 	utility: 1.020384
Epoch 800: 	avg bid: 1.006107, 	utility: 0.695619
Epoch 850: 	avg bid: 1.044088, 	utility: 1.155876
Epoch 900: 	avg bid: 0.982303, 	utility: 1.224215
Epoch 950: 	avg bid: 0.907457, 	utility: 1.273471
Epo